# 🎬 Generador de Vídeos 60s para 60secondstrip

Este notebook genera vídeos de 60 segundos para cada POI usando:
- **MoviePy** para edición de vídeo
- **gTTS** para síntesis de voz
- **PIL** para manipulación de imágenes
- **Efecto Ken Burns** para dinamismo visual

## 📁 Estructura de archivos:
- `input/`: Scripts y datos de entrada
- `images/`: Imágenes de POIs
- `output/`: Vídeos generados
- `audio/`: Archivos de audio TTS


In [ ]:
import os
import io
import math
import tempfile
import textwrap
import hashlib
from typing import List, Optional
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
import requests
import json

# Configuración
INPUT_DIR = 'input'
IMAGES_DIR = 'images'
OUTPUT_DIR = 'output'
AUDIO_DIR = 'audio'

# Crear directorios si no existen
for dir_path in [INPUT_DIR, IMAGES_DIR, OUTPUT_DIR, AUDIO_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print("✅ Directorios creados")


## ⚡ Configuración para vídeos RÁPIDOS

- **Duración:** 60 segundos exactos
- **Estilo:** Dinámico, directo, sin rollos
- **Imágenes:** 3-4 máximo por vídeo
- **Texto:** Frases cortas y directas
- **Efectos:** Ken Burns suave + transiciones rápidas


In [ ]:
def build_fast_video(script_lines: List[str], images: List[str], out_name: str = 'video_60s.mp4', lang: str = 'es') -> str:
    """Genera vídeo RÁPIDO de 60 segundos - sin rollos"""
    try:
        print(f"⚡ Generando vídeo RÁPIDO: {out_name}")
        
        # 1) Generar TTS
        narration_text = ' '.join(script_lines)
        audio_path = synthesize_tts(narration_text, lang=lang, slow=False)  # Más rápido
        
        if not audio_path:
            print("❌ No se pudo generar audio")
            return None
        
        audio = AudioFileClip(audio_path)
        target_total = 60.0
        
        # 2) Solo 3-4 imágenes máximo para velocidad
        n = min(len(images), 4)
        per_slide = target_total / n
        
        print(f"⚡ {n} slides de {per_slide:.1f}s - SÚPER RÁPIDO")
        
        # 3) Preparar imágenes
        local_images = []
        for img_url in images[:n]:
            if img_url.startswith('http'):
                local_path = download_image(img_url)
                if local_path:
                    local_images.append(local_path)
            else:
                local_images.append(img_url)
        
        # Crear placeholders si faltan
        while len(local_images) < n:
            placeholder_text = f"Slide {len(local_images) + 1}"
            placeholder_path = create_placeholder_image(placeholder_text)
            local_images.append(placeholder_path)
        
        # 4) Ken Burns MÁS SUAVE para velocidad
        img_clips = []
        for i, img_path in enumerate(local_images):
            clip = ken_burns_zoom(img_path, per_slide, zoom_factor=1.05)  # Zoom más suave
            clip = add_overlay_gradient(clip)
            img_clips.append(clip)
        
        # 5) Subtítulos MÁS GRANDES y directos
        lines_per_slide = max(1, math.ceil(len(script_lines) / n))
        subs = []
        
        for i in range(n):
            start_idx = i * lines_per_slide
            end_idx = min((i + 1) * lines_per_slide, len(script_lines))
            chunk = script_lines[start_idx:end_idx]
            
            if not chunk:
                chunk = [script_lines[min(i, len(script_lines)-1)]]
            
            sub_text = ' '.join(chunk)
            sub = create_subtitle(sub_text, per_slide, font_size=80)  # Texto más grande
            subs.append(sub)
        
        # 6) Transiciones RÁPIDAS
        img_clips = [c.crossfadein(0.2).crossfadeout(0.2) for c in img_clips]  # Más rápido
        
        # 7) Componer vídeo
        composed = []
        for img_clip, sub_clip in zip(img_clips, subs):
            composed.append(CompositeVideoClip([img_clip, sub_clip]))
        
        video = concatenate_videoclips(composed, method='compose')
        video = video.set_audio(audio).set_fps(30)
        
        # 8) Exportar RÁPIDO
        out_path = os.path.join(OUTPUT_DIR, out_name)
        print(f"💾 Exportando RÁPIDO: {out_path}")
        
        video.write_videofile(
            out_path,
            codec='libx264',
            audio_codec='aac',
            threads=4,  # Más hilos
            preset='ultrafast',  # Preset más rápido
            bitrate='2000k',  # Bitrate menor para velocidad
            verbose=False,
            logger=None
        )
        
        print(f"✅ Vídeo RÁPIDO generado: {out_name}")
        return out_path
        
    except Exception as e:
        print(f"❌ Error generando vídeo: {e}")
        return None


In [ ]:
# Función para procesar POIs RÁPIDO
def process_fast_poi_videos(manifest_path: str = '../public/content/manifest.json'):
    """Procesa todos los POIs del manifest y genera vídeos RÁPIDOS"""
    try:
        # Cargar manifest
        with open(manifest_path, 'r', encoding='utf-8') as f:
            manifest = json.load(f)
        
        print(f"⚡ Procesando RÁPIDO {len(manifest['cities'])} ciudades")
        
        for city in manifest['cities']:
            print(f"\n🏛️ Ciudad: {city['name']}")
            
            for poi in city['pois']:
                print(f"\n📍 POI: {poi['name']}")
                
                # Generar vídeo para adultos - RÁPIDO
                if poi.get('summaryAdult'):
                    script_adult = poi['summaryAdult'].split('. ')
                    script_adult = [line.strip() + '.' for line in script_adult if line.strip()]
                    
                    video_name_adult = f"{poi['id']}_adult.mp4"
                    build_fast_video(
                        script_adult,
                        [poi['imageUrl']],
                        video_name_adult,
                        'es'
                    )
                
                # Generar vídeo para niños - RÁPIDO
                if poi.get('summaryKids'):
                    script_kids = poi['summaryKids'].split('. ')
                    script_kids = [line.strip() + '.' for line in script_kids if line.strip()]
                    
                    video_name_kids = f"{poi['id']}_kids.mp4"
                    build_fast_video(
                        script_kids,
                        [poi['imageUrl']],
                        video_name_kids,
                        'es'
                    )
        
        print("\n🎉 Procesamiento RÁPIDO completado")
        
    except Exception as e:
        print(f"❌ Error procesando manifest: {e}")

# Ejemplo de uso individual RÁPIDO
def generate_fast_poi_video(poi_id: str, summary: str, image_url: str, is_kids: bool = False):
    """Genera vídeo RÁPIDO para un POI específico"""
    script = summary.split('. ')
    script = [line.strip() + '.' for line in script if line.strip()]
    
    video_name = f"{poi_id}_{'kids' if is_kids else 'adult'}.mp4"
    
    return build_fast_video(script, [image_url], video_name, 'es')


In [ ]:
# 🚀 EJECUTAR GENERACIÓN RÁPIDA
process_fast_poi_videos()
